# Retriever


## Buat Haystack Pipeline untuk retrieve data dari mongodb atlas

In [ ]:
import os
from getpass import getpass
os.environ["MONGO_CONNECTION_STRING"] = getpass("Masukkan MongoDB Connection String Anda: ")

In [16]:
from haystack import Pipeline
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack_integrations.document_stores.mongodb_atlas import MongoDBAtlasDocumentStore
from haystack_integrations.components.retrievers.mongodb_atlas import MongoDBAtlasEmbeddingRetriever

document_store = MongoDBAtlasDocumentStore(
    database_name="depato_store",
    collection_name="products",
    vector_search_index="vector_index",
    full_text_search_index="search_index",
)

In [17]:
pipeline = Pipeline()
pipeline.add_component("embedder", SentenceTransformersTextEmbedder())
pipeline.add_component("retriever", MongoDBAtlasEmbeddingRetriever(document_store=document_store,top_k=10))

In [18]:
pipeline.connect("embedder", "retriever")

🚅 Components
  - embedder: SentenceTransformersTextEmbedder
  - retriever: MongoDBAtlasEmbeddingRetriever
🛤️ Connections
  - embedder.embedding -> retriever.query_embedding (List[float])

In [24]:
pipeline.run(
    {
        "embedder":{
            "text":"comfortable dress for going out with friends"
        }
    }
)

Batches: 100%|██████████| 1/1 [00:00<00:00, 15.91it/s]


{'retriever': {'documents': [Document(id=1e51b2fdd4606b16cfb582e64ffd7fb44e36fa2423fed1cf0f02ad5396015c3f, content: 'Rasta Imposta Flag Dress USA
    The usa flag dress is the best outfit to wear if you want to show off ...', meta: {'asin': 'B00C3DLHY0', 'title': 'Rasta Imposta Flag Dress USA', 'brand': 'Rasta Imposta', 'price': 24.33, 'gender': 'female', 'material': 'Polyester', 'category': 'Dresses/Jumpsuits'}, score: 0.7217811942100525, embedding: vector of size 768),
   Document(id=acbb5d5fde8d26791049fa83522c8fc41b1c1a0990db1b9be9f4ef922da3efce, content: 'Style J Sweet Grace Denim Skirt
    Caual style long skirt, great price and high quality', meta: {'asin': 'B01AO9TSBS', 'title': 'Style J Sweet Grace Denim Skirt', 'brand': 'Style J', 'price': 35.0, 'gender': 'female', 'material': 'Cotton', 'category': 'Bottoms'}, score: 0.7141729593276978, embedding: vector of size 768),
   Document(id=95cd4b8bb0a6182ca8aa1a1b7ab3da594d0b3c900f2db6770e6e856fd8e1c494, content: 'Style J Flowing Ch

## Menggunakan Meta Data untuk memfilter

### Single Filter

In [25]:
filters = {
    "field":"meta.category","operator":"==","value":"Dresses/Jumpsuits"
}

In [27]:
pipeline.run(
    {
        "embedder":{
            "text":"comfortable dress for going out with friends"
        },
        "retriever":{
            "filters": filters
        }
    }
)

Batches: 100%|██████████| 1/1 [00:00<00:00, 11.28it/s]


{'retriever': {'documents': [Document(id=1e51b2fdd4606b16cfb582e64ffd7fb44e36fa2423fed1cf0f02ad5396015c3f, content: 'Rasta Imposta Flag Dress USA
    The usa flag dress is the best outfit to wear if you want to show off ...', meta: {'asin': 'B00C3DLHY0', 'title': 'Rasta Imposta Flag Dress USA', 'brand': 'Rasta Imposta', 'price': 24.33, 'gender': 'female', 'material': 'Polyester', 'category': 'Dresses/Jumpsuits'}, score: 0.7217811942100525, embedding: vector of size 768),
   Document(id=13a473e9ef572dbfe8f37cef1f1b59e02d43394e90addbe598c83ce4560117fd, content: 'Ivanka Trump Women's Lace Dress
    Sleeveless floral dress with blush underlay perfect to we at for a ...', meta: {'asin': 'B01FH59HXS', 'title': "Ivanka Trump Women's Lace Dress", 'brand': nan, 'price': 98.6, 'gender': 'female', 'material': 'Polyester', 'category': 'Dresses/Jumpsuits'}, score: 0.7054650187492371, embedding: vector of size 768),
   Document(id=c72b53a22948126b5860fd58d59593a09e23dbca93666fcb36e6c4b605106a8c, con

### Multiple Filter

In [32]:
filters = {
    "operator":"AND",
    "conditions":[
        {"field":"meta.category","operator":"==","value":"Dresses/Jumpsuits"},
        {"field":"meta.price", "operator":"<=", "value":50},
    ]
    
}

In [33]:
pipeline.run(
    {
        "embedder":{
            "text":"comfortable dress for going out with friends"
        },
        "retriever":{
            "filters": filters
        }
    }
)

Batches: 100%|██████████| 1/1 [00:00<00:00, 10.68it/s]


{'retriever': {'documents': [Document(id=1e51b2fdd4606b16cfb582e64ffd7fb44e36fa2423fed1cf0f02ad5396015c3f, content: 'Rasta Imposta Flag Dress USA
    The usa flag dress is the best outfit to wear if you want to show off ...', meta: {'asin': 'B00C3DLHY0', 'title': 'Rasta Imposta Flag Dress USA', 'brand': 'Rasta Imposta', 'price': 24.33, 'gender': 'female', 'material': 'Polyester', 'category': 'Dresses/Jumpsuits'}, score: 0.7217811942100525, embedding: vector of size 768),
   Document(id=a5c3efc5fa4f4c86d8c81dabb74b64c03020687d84be462053aaefc2f94d58cb, content: 'A. Byer Juniors' Cable-Knit Fashion Sweater Dress
    Soft knit styling sweater dress with deep cowl ne...', meta: {'asin': 'B011T4NTU8', 'title': "A. Byer Juniors' Cable-Knit Fashion Sweater Dress", 'brand': nan, 'price': 34.99, 'gender': 'female', 'material': 'Polyester', 'category': 'Dresses/Jumpsuits'}, score: 0.6889418363571167, embedding: vector of size 768),
   Document(id=db38bf21b88afde79cc0300a25915261b8d75f020d354b81d8

### Advanced Filtering

In [34]:
filters = {
    "operator":"AND",
    "conditions":[
        {"field":"meta.category","operator":"==","value":"Dresses/Jumpsuits"},
        {"field":"meta.price", "operator":"<=", "value":50},
        {
            "operator":"OR",
            "conditions":[
                {"field":"meta.material","operator":"==","value":"Cotton"},
                {"field":"meta.material","operator":"==","value":"Polyester"}
            ]
        }
    ]
    
}

In [35]:
pipeline.run(
    {
        "embedder":{
            "text":"comfortable dress for going out with friends"
        },
        "retriever":{
            "filters": filters
        }
    }
)

Batches: 100%|██████████| 1/1 [00:00<00:00, 14.23it/s]


{'retriever': {'documents': [Document(id=1e51b2fdd4606b16cfb582e64ffd7fb44e36fa2423fed1cf0f02ad5396015c3f, content: 'Rasta Imposta Flag Dress USA
    The usa flag dress is the best outfit to wear if you want to show off ...', meta: {'asin': 'B00C3DLHY0', 'title': 'Rasta Imposta Flag Dress USA', 'brand': 'Rasta Imposta', 'price': 24.33, 'gender': 'female', 'material': 'Polyester', 'category': 'Dresses/Jumpsuits'}, score: 0.7217811942100525, embedding: vector of size 768),
   Document(id=a5c3efc5fa4f4c86d8c81dabb74b64c03020687d84be462053aaefc2f94d58cb, content: 'A. Byer Juniors' Cable-Knit Fashion Sweater Dress
    Soft knit styling sweater dress with deep cowl ne...', meta: {'asin': 'B011T4NTU8', 'title': "A. Byer Juniors' Cable-Knit Fashion Sweater Dress", 'brand': nan, 'price': 34.99, 'gender': 'female', 'material': 'Polyester', 'category': 'Dresses/Jumpsuits'}, score: 0.6889418363571167, embedding: vector of size 768),
   Document(id=db38bf21b88afde79cc0300a25915261b8d75f020d354b81d8